In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/prodazha-pomescheniy?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

465


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 11/11 [00:03<00:00,  3.01it/s]

439


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Покупка помещений'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 439/439 [01:56<00:00,  3.78it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этаж / этажность,Высота потолков,...,Интернет,Торг уместен,Телефон,Мебель,Этажность,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/prodazha-pomescheniy/ul...,"Продажа помещения на ул. Родионова, д. 192Ниже...",39 999 999 руб.,Продам офисный блок в Бизнесцентре на улице Р...,2860084,Нижегородский,"ул. Родионова, д. 192",815 м²,2 / 9,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Покупка помещений,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/prodazha-pomescheniy/ul...,"Продажа помещения на ул. Максима Горького, д. ...",30 000 000 руб.,Продаю нежилое помещение.\n\nАдрес: Нижний Нов...,2788351,Нижегородский,"ул. Максима Горького, д. 152А","265,7 м²",1 / 9,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Покупка помещений,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/prodazha-pomescheniy/ul...,"Продажа помещения на ул. Верхнепечёрская, д. 7...",80 000 000 руб.,Здание приносит аренду 700 тыс.руб. Три стабил...,2802080,Нижегородский,"ул. Верхнепечёрская, д. 7Д","928,4 м²",2 / 2,3 м,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Покупка помещений,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/prodazha-pomescheniy/ul...,"Продажа помещения на ул. Памирская, д. 11Ленин...",66 000 000 руб.,"Ленинский район, улица Памирская, Баумана, Кос...",2867942,Ленинский,"ул. Памирская, д. 11",2 200 м²,1 / 4,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Покупка помещений,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/prodazha-pomescheniy/ul...,"Продажа помещения на ул. Максима Горького, д. ...",3 500 000 руб.,"Отличное местоположение, Центр города, подвал,...",2862279,Нижегородский,"ул. Максима Горького, д. 232",84 м²,-1 / 10,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Покупка помещений,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('indoor-buy.xlsx', index=False)